In [2]:
import pandas as pd
import numpy as np
import sqlite3
import altair as alt
import matplotlib.pyplot as plt

alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [3]:
con = sqlite3.connect("switrs.sqlite")

query = """
    SELECT * FROM collisions WHERE county_location = 'los angeles'
    """

df = pd.read_sql_query(query, con, parse_dates = ["collision_date"])
df["year"] = df["collision_date"].dt.year
df = df.query("year < 2021") # remove incomplete 2021 data
df["alcohol_involved"] = df["alcohol_involved"].fillna(0) # convert NaN to 0 in alcohol use column

dfc = df[["case_id", "county_location", "alcohol_involved", "collision_severity", "injured_victims", "collision_date", "year", "collision_time", "party_count"]]

#### Distribution of the proportion of crashes involving alcohol by time of day

In [ ]:
# We will attempt to use multiple models to fit this distribution. We will use polynomial splines and B-splines but it appears a skewed-normal may be a possible fit as well.

#### Distributions of the number of injuries per collision, alcohol vs. no alcohol

In [ ]:
# These appear to follow exponential or Weibull distributions. We will determine which is the best fit and then compare to see if alcohol has an effect on the distribution.

#### Timeline for number of collisions

In [ ]:
# train on up to 2019, predict 2020, compare to actual. This will be done using a time series model.

#### Timeline for number of collisions (alcohol vs no alcohol)

In [ ]:
# trained on up to 2019, predicting 2020, but separating by alcohol vs. no alcohol and comparing to actuals. This will also be completed using a time series model

#### Feature selection using LASSO on fitted GLM against several labels

#### Feature selection using Random Forest feature importance against several labels